<a href="https://colab.research.google.com/github/pgallino/Organizacion-de-datos-TP1-1C2023/blob/main/spark34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/logs.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


34 ¿Quién es el usuario que más ha bloqueado a otros? (⭐)

In [ ]:
rdd.count()

12203103

In [ ]:
rdd.take(5)

[Row(item_id='1', timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id='2', timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id='3', timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id='4', timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id='7856', contributor_ip=None, comment='Sin información de orige

Me quedo con los registros de acción block.

In [ ]:
rdd_blocks = rdd.filter(lambda x: x.action == 'block')

In [ ]:
rdd_blocks.count()

342277

Genero registros de la forma (username, 1) Para contar cuantos bloqueó cada usuario.

In [ ]:
rdd_blocks = rdd_blocks.map(lambda x: (x.contributor_username, 1))

Con reduceByKey, sumo por key la cantidad de bloqueos.

In [ ]:
rdd_blocks = rdd_blocks.reduceByKey(lambda x,y : x+y)

Tomo ordenados los primeros 10.

In [ ]:
rdd_blocks.takeOrdered(10, lambda x: -x[1])

[('Magister Mathematicae', 29226),
 ('Montgomery', 18675),
 ('Taichi', 15833),
 ('LuchoCR', 13268),
 ('Antur', 12600),
 ('Ontzak', 11987),
 ('Marcelo', 9927),
 ('Netito777', 9212),
 ('Geom', 8955),
 ('RoyFocker', 8856)]

Por lo que podemos ver que 'Magister Mathematicae' es el usuario que más bloqueo a otros, con 29226 bloqueos.